# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [29]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0

for sqrt_value in generator:
    print(f'Returned {sqrt_value}')
    sum += sqrt_value

print(sum)

Returned 1.0
Returned 1.4142135623730951
Returned 1.7320508075688772
Returned 2.0
Returned 2.23606797749979
8.382332347441762


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [12]:
import dlt

sum = 0
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)
    sum += person["Age"]

print(sum)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)
    sum += person["Age"]

print(sum)

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_table",
										write_disposition="append")

info = generators_pipeline.run(people_2(),
										table_name="people_table",
										write_disposition="append")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
140
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
353


In [14]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_table below:")

table = conn.sql("SELECT * FROM people_table").df()
display(table)

sum_sql = conn.sql("SELECT SUM(AGE) FROM people_table").df()
display(sum_sql)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
│ people_table        │
│ stream_download     │
└─────────────────────┘




 people_table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707980526.8244138,VyoE4ZFDn+Wn8A,None
1,2,Person_2,27,City_A,1707980526.8244138,aqCQ3cZuXmUL7w,None
2,3,Person_3,28,City_A,1707980526.8244138,mYESzzrGRpO1Lg,None
3,4,Person_4,29,City_A,1707980526.8244138,4jzlnrfL5hboew,None
4,5,Person_5,30,City_A,1707980526.8244138,63CJRlo6UdVSmA,None
5,3,Person_3,33,City_B,1707980527.83166,uU88hmVFWaqNiw,Job_3
6,4,Person_4,34,City_B,1707980527.83166,vKwxAG0qGRu2zg,Job_4
7,5,Person_5,35,City_B,1707980527.83166,/+sdmQHGe9pCug,Job_5
8,6,Person_6,36,City_B,1707980527.83166,ZlA1uJaa3pdizQ,Job_6
9,7,Person_7,37,City_B,1707980527.83166,TByne30B+eZl0w,Job_7


,sum(AGE)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [28]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
drop_table = conn.sql("DROP TABLE IF EXISTS merge_people_table;")
display(drop_table)

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
										table_name="merge_people_table",
										write_disposition="merge",
                                        primary_key="ID")

info = generators_pipeline.run(people_2(),
										table_name="merge_people_table",
										write_disposition="merge",
                                        primary_key="ID")

print(info)


conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n merge_people_table below:")

table = conn.sql("SELECT * FROM merge_people_table").df()
display(table)

sum_sql = conn.sql("SELECT SUM(AGE) FROM merge_people_table").df()
display(sum_sql)

None

Pipeline dlt_ipykernel_launcher load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////workspaces/zoomcamp2024/mage-zoomcamp/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707982669.669952 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
│ merge2_people_table │
│ merge_people_table  │
│ people_table        │
│ stream_download     │
└─────────────────────┘




 merge_people_table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707982668.7767599,OOJ8tWFMCIrgsA,None
1,1,Person_1,26,City_A,1707982668.7767599,xAop5qmIIVE6yw,None
2,5,Person_5,35,City_B,1707982669.669952,HUf9FtuuD2cOGw,Job_5
3,8,Person_8,38,City_B,1707982669.669952,NwOdF2BydKeD7Q,Job_8
4,7,Person_7,37,City_B,1707982669.669952,M9dQgkSJ7lF3zg,Job_7
5,4,Person_4,34,City_B,1707982669.669952,h5ZC3f/1OPKqEA,Job_4
6,3,Person_3,33,City_B,1707982669.669952,877gnzJau4shGA,Job_3
7,6,Person_6,36,City_B,1707982669.669952,+QwivMElo0DYAQ,Job_6


,sum(AGE)
0,266.0


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX